In [ ]:
from google.colab import drive
import numpy as np

# Mount Google Drive
drive.mount('/content/drive')

# Load ECG signals and labels from your Google Drive (update path as needed)
ecg_signals = np.load('/content/drive/MyDrive/ECG_Project/ecg_signals.npy')
ecg_labels = np.load('/content/drive/MyDrive/ECG_Project/ecg_labels.npy')

print(f"ECG Signals Shape: {ecg_signals.shape}")
print(f"ECG Labels Shape: {ecg_labels.shape}")


In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.fftpack import fft
from scipy.signal import find_peaks

# Function to extract features from a single ECG signal
def extract_features(ecg_signal):
    features = {}

    # Statistical features
    features['mean'] = np.mean(ecg_signal)
    features['variance'] = np.var(ecg_signal)
    features['kurtosis'] = stats.kurtosis(ecg_signal)
    features['skewness'] = stats.skew(ecg_signal)

    # Frequency domain features (e.g., Spectral Entropy)
    spectral_entropy = -np.sum(fft(ecg_signal) * np.log2(fft(ecg_signal)))
    features['spectral_entropy'] = spectral_entropy

    # Morphological features (RR intervals, SDNN, RMSSD)
    peaks, _ = find_peaks(ecg_signal, distance=300)  # Example peak detection
    rr_intervals = np.diff(peaks)
    features['mean_rr'] = np.mean(rr_intervals)
    features['sdnn'] = np.std(rr_intervals)
    features['rmssd'] = np.sqrt(np.mean(np.square(np.diff(rr_intervals))))

    return features

# Process multiple ECG signals and save the features to a CSV
def process_ecg_dataset(ecg_signals, labels, output_file='ecg_features.csv'):
    features_list = []

    for i, ecg_signal in enumerate(ecg_signals):
        features = extract_features(ecg_signal)
        features['label'] = labels[i]
        features_list.append(features)

    features_df = pd.DataFrame(features_list)
    features_df.to_csv(output_file, index=False)
    print(f"Features saved to {output_file}")

# Example usage
if __name__ == "__main__":
    process_ecg_dataset(ecg_signals, ecg_labels, output_file='/content/ecg_features.csv')


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
import pandas as pd

# Load the extracted features dataset
data = pd.read_csv('/content/ecg_features.csv')
X = data.drop('label', axis=1).values  # Features
y = data['label'].values  # Labels (0: low stress, 1: medium stress, 2: high stress)

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the data for CNN input
X_train = X_train.reshape(-1, X_train.shape[1], 1)
X_val = X_val.reshape(-1, X_val.shape[1], 1)

# Define the CNN model
def create_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Create the model
input_shape = (X_train.shape[1], 1)
model = create_cnn_model(input_shape)

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=32)

# Save the model
model.save('/content/stress_recognition_model.h5')

print("Training Complete")



In [ ]:
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pandas as pd

# Load the trained model
model = tf.keras.models.load_model('/content/stress_recognition_model.h5')

# Load the test dataset
test_data = pd.read_csv('/content/ecg_test_features.csv')  # Ensure you have the test feature file
X_test = test_data.drop('label', axis=1).values
y_test = test_data['label'].values

# Reshape the test data for the CNN model
X_test = X_test.reshape(-1, X_test.shape[1], 1)

# Make predictions
y_pred = model.predict(X_test)
y_pred = y_pred.argmax(axis=1)  # Get the class with highest probability

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=["Low Stress", "Medium Stress", "High Stress"]))

# Print confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Load test results (Predictions vs Ground Truth)
results = pd.read_csv('/content/test_results.csv')  # Columns ['y_true', 'y_pred']

# Plot Confusion Matrix
def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Low Stress', 'Medium Stress', 'High Stress'], yticklabels=['Low Stress', 'Medium Stress', 'High Stress'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

# Plot Feature Boxplots
def plot_feature_boxplots():
    data = pd.read_csv('/content/ecg_features.csv')  # Ensure you have the feature file
    plt.figure(figsize=(12, 8))
    sns.boxplot(data=data.drop('label', axis=1))
    plt.title('Boxplot of ECG Features')
    plt.xticks(rotation=90)
    plt.show()

# Classification report
def classification_analysis(y_true, y_pred):
    print(classification_report(y_true, y_pred, target_names=['Low Stress', 'Medium Stress', 'High Stress']))

if __name__ == "__main__":
    y_true = results['y_true']
    y_pred = results['y_pred']

    plot_confusion_matrix(y_true, y_pred)
    plot_feature_boxplots()
    classification_analysis(y_true, y_pred)
